# Production CODE

In [209]:
import pandas as pd
import numpy as np

# ใส่ file ที่เรา upload เข้าไป
credit = '/content/database for credit limit.xlsx'
opus = '/content/opus enquiry.xlsx'


####################################################### SOURCE CODE ##############################################################################################
df_credit = pd.read_excel(credit)
df_opus = pd.read_excel(opus)
#select only specific columns
df_opus = df_opus[['Customer Code', 'Customer Name', 'Amount (THB)', 'Aging Sort']]

#checking parent and child company with credit master file
df_opus_new = df_opus.merge(df_credit, left_on='Customer Code', right_on='child code')[['Customer Code', 'Customer Name', 'Amount (THB)', 'Aging Sort',
                                                                        'Parent Customer Code', 'Parent Customer Name']].copy()

# grouping parent company
df_opus_new_group = df_opus_new.groupby(['Parent Customer Code', 'Parent Customer Name', 'Aging Sort'])['Amount (THB)'].sum().reset_index().copy()

# pivot columns "Aging Sort" after pivot columns, we will lose column Parent Customer Name but we will get it back later
df_opus_new_group_pivot = df_opus_new_group.pivot(index='Parent Customer Code', columns='Aging Sort', values=['Amount (THB)']).copy()

# set multilevel from pandas pivot to be one level dataframe
df_opus_new_group_pivot.columns=['01-07 days','08-15 days','16-30 days','31-60 days','61-90 days','Not due']
df_opus_new_group_pivot.reset_index(inplace=True)

# getting columns Parent Customer Name from credit master file
df_final = df_opus_new_group_pivot.merge(df_credit, left_on='Parent Customer Code', right_on='Parent Customer Code', how='left').iloc[:, :9]

# after we get Parent Customer Name back, it will duplicated since one Parent name have many child name so we have to drop duplicated
df_final = df_final.drop_duplicates(subset='Parent Customer Code')
df_final.rename(columns={
    'Parent Customer Code': 'Customer Code',
    'Parent Customer Name': 'Customer Name',
    'Parent Credit Limit': 'Credit Limit'
}, inplace=True)

# aggregate new columns for making final report
df_final = df_final.fillna(0)
df_final['over due'] = df_final['01-07 days'] + df_final['08-15 days'] + df_final['16-30 days'] + df_final['31-60 days'] + df_final['61-90 days']
df_final['outstanding'] = df_final['over due'] + df_final['Not due']
df_final['%'] = df_final['outstanding']/df_final['Credit Limit']*100
df_final = df_final.sort_values(by='%', ascending=False)

# re-order columns
col = ['Customer Code', 'Customer Name', 'Credit Limit', '%', 'Not due', '01-07 days', '08-15 days',
       '16-30 days', '31-60 days', '61-90 days', 'over due', 'outstanding']
df_final = df_final.reindex(columns=col)

# save output in excel format
df_final.to_excel('utilize credit limit.xlsx', index=False)

# Code for Debug

In [1]:
import pandas as pd
import numpy as np

In [48]:
df_credit = pd.read_excel('/content/database for credit limit.xlsx')
df_opus = pd.read_excel('/content/opus enquiry.xlsx')


df_opus = df_opus[['Customer Code', 'Customer Name', 'Amount (THB)', 'Aging Sort']]

In [52]:
print('OPUS')
print(f'row : {df_opus.shape[0]}')
print(f'columns : {df_opus.shape[1]}')

OPUS
row : 14856
columns : 4


In [53]:
df_credit.head(5)

,Parent Customer Code,Parent Customer Name,Parent Credit Limit,child code,Child Name,Credit Start Date,Credit End Date
0,TH100210,"AJINOMOTO CO., (THAILAND) LTD.",16800,TH100210,"AJINOMOTO CO., (THAILAND) LTD.",2024-07-01,2025-06-30
1,TH100268,ALUCON PUBLIC COMPANY LIMITED,26229,TH100268,ALUCON PUBLIC COMPANY LIMITED,2024-07-01,2025-06-30
2,TH100347,"KERRY-APEX (THAILAND) CO., LTD.",558000,TH100347,"KERRY-APEX (THAILAND) CO., LTD.",2024-05-01,2025-04-30
3,TH100477,"ASIAN HONDA MOTOR CO.,LTD.",634757,TH102550,"HONDA AUTOMOBILE (THAILAND) CO.,LTD",2024-06-01,2025-05-31
4,TH100477,"ASIAN HONDA MOTOR CO.,LTD.",634757,TH106782,"THAI HONDA CO., LTD.",2024-06-01,2025-05-31


In [54]:
df_credit['child code'].duplicated().sum()

0

In [55]:
df_opus.merge(df_credit, left_on='Customer Code', right_on='child code').columns

Index(['Customer Code', 'Customer Name', 'Amount (THB)', 'Aging Sort',
       'Parent Customer Code', 'Parent Customer Name', 'Parent Credit Limit',
       'child code', 'Child Name', 'Credit Start Date', 'Credit End Date'],
      dtype='object')

In [66]:
df_opus_new = df_opus.merge(df_credit, left_on='Customer Code', right_on='child code')[['Customer Code', 'Customer Name', 'Amount (THB)', 'Aging Sort',
                                                                        'Parent Customer Code', 'Parent Customer Name']].copy()

In [67]:
df_opus_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14853 entries, 0 to 14852
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Customer Code         14853 non-null  object 
 1   Customer Name         14853 non-null  object 
 2   Amount (THB)          14853 non-null  float64
 3   Aging Sort            14853 non-null  object 
 4   Parent Customer Code  14853 non-null  object 
 5   Parent Customer Name  14853 non-null  object 
dtypes: float64(1), object(5)
memory usage: 696.4+ KB


In [68]:
# checking the 3 missing row which is not found in credit master
df_opus[~df_opus['Customer Code'].isin(df_opus_new['Customer Code'])]

,Customer Code,Customer Name,Amount (THB),Aging Sort
2570,CH100404,IKEA SUPPLY AG,3700.00,Not due
6866,CH100404,IKEA SUPPLY AG,57200.00,Not due
8519,TH107669,"UNITED THAI LOGISTICS CO., LTD.",180111.05,Not due


In [69]:
df_opus_new.sample(5)

,Customer Code,Customer Name,Amount (THB),Aging Sort,Parent Customer Code,Parent Customer Name
8786,TH101735,"ECU WORLDWIDE (THAILAND) CO., LTD.",1717.91,Not due,TH101735,"ECU WORLDWIDE (THAILAND) CO., LTD."
3496,TH108506,"LX PANTOS (THAILAND) CO.,LTD",800.00,Not due,TH108506,"LX PANTOS (THAILAND) CO.,LTD"
14206,TH105539,"SANKYU-THAI CO.,LTD.",18171.49,Not due,TH108568,"SANKYU-THAI CO.,LTD."
799,TH518330,"TOYOTA MOTOR ASIA (THAILAND) CO., LTD.",20610.54,Not due,TH113675,TOYOTA DAIHATSU ENGINEERING & MANUFACTURING CO...
14600,TH516570,SCGC ICO POLYMERS COMPANY LIMITED,6723.84,Not due,TH106926,"THAI POLYETHYLENE CO.,LTD."


In [73]:
df_opus_new_group = df_opus_new.groupby(['Parent Customer Code', 'Parent Customer Name', 'Aging Sort'])['Amount (THB)'].sum().reset_index().copy()
df_opus_new_group.head(7)

,Parent Customer Code,Parent Customer Name,Aging Sort,Amount (THB)
0,TH100210,"AJINOMOTO CO., (THAILAND) LTD.",Not due,161805.36
1,TH100268,ALUCON PUBLIC COMPANY LIMITED,Not due,326154.49
2,TH100347,"KERRY-APEX (THAILAND) CO., LTD.",08-15 days,10908.00
3,TH100347,"KERRY-APEX (THAILAND) CO., LTD.",Not due,15165662.70
4,TH100477,"ASIAN HONDA MOTOR CO.,LTD.",01-07 days,1097602.98
5,TH100477,"ASIAN HONDA MOTOR CO.,LTD.",08-15 days,188500.00
6,TH100477,"ASIAN HONDA MOTOR CO.,LTD.",31-60 days,5500.00


In [74]:
print('OPUS after grouping')
print(f'row : {df_opus_new_group.shape[0]}')
print(f'columns : {df_opus_new_group.shape[1]}')

OPUS after grouping
row : 301
columns : 4


In [133]:
df_opus_new_group_pivot = df_opus_new_group.pivot(index='Parent Customer Code', columns='Aging Sort', values=['Amount (THB)']).copy()
df_opus_new_group_pivot

Amount (THB)                                              \
Aging Sort             01-07 days 08-15 days 16-30 days 31-60 days 61-90 days   
Parent Customer Code                                                            
TH100210                      NaN        NaN        NaN        NaN        NaN   
TH100268                      NaN        NaN        NaN        NaN        NaN   
TH100347                      NaN    10908.0        NaN        NaN        NaN   
TH100477               1097602.98   188500.0        NaN     5500.0        NaN   
TH100707                      NaN        NaN        NaN        NaN        NaN   
...                           ...        ...        ...        ...        ...   
TH511109                  5000.00        NaN        NaN        NaN        NaN   
TH513541                      NaN        NaN        NaN        NaN        NaN   
TH513953                      NaN        NaN        NaN        NaN        NaN   
TH515918                      NaN        NaN        NaN        NaN        NaN   
TH516584                  7300.00        NaN      800.0        NaN        NaN   

                                   
Aging Sort                Not due  
Parent Customer Code               
TH100210                161805.36  
TH100268                326154.49  
TH100347              15165662.70  
TH100477              13679531.94  
TH100707                 72515.25  
...                           ...  
TH511109              11610062.64  
TH513541                 72133.46  
TH513953                314890.36  
TH515918                672859.86  
TH516584              13156886.45  

[132 rows x 6 columns]

In [139]:
df_opus_new_group_pivot.columns=['01-07 days','08-15 days','16-30 days','31-60 days','61-90 days','Not due']
df_opus_new_group_pivot.reset_index(inplace=True)

In [140]:
df_opus_new_group_pivot

,Parent Customer Code,01-07 days,08-15 days,16-30 days,31-60 days,61-90 days,Not due
0,TH100210,NaN,NaN,NaN,NaN,NaN,161805.36
1,TH100268,NaN,NaN,NaN,NaN,NaN,326154.49
2,TH100347,NaN,10908.0,NaN,NaN,NaN,15165662.70
3,TH100477,1097602.98,188500.0,NaN,5500.0,NaN,13679531.94
4,TH100707,NaN,NaN,NaN,NaN,NaN,72515.25
...,...,...,...,...,...,...,...
127,TH511109,5000.00,NaN,NaN,NaN,NaN,11610062.64
128,TH513541,NaN,NaN,NaN,NaN,NaN,72133.46
129,TH513953,NaN,NaN,NaN,NaN,NaN,314890.36
130,TH515918,NaN,NaN,NaN,NaN,NaN,672859.86


In [177]:
df_opus_new_group_pivot.iloc[:, 1:].sum()

,0
01-07 days,5.157882e+07
08-15 days,2.895664e+07
16-30 days,5.939191e+06
31-60 days,1.775577e+05
61-90 days,2.320711e+04
Not due,7.528365e+08


In [199]:
df_final = df_opus_new_group_pivot.merge(df_credit, left_on='Parent Customer Code', right_on='Parent Customer Code', how='left').iloc[:, :9]
df_final = df_final.drop_duplicates(subset='Parent Customer Code')
df_final

,Parent Customer Code,01-07 days,08-15 days,16-30 days,31-60 days,61-90 days,Not due,Parent Customer Name,Parent Credit Limit
0,TH100210,NaN,NaN,NaN,NaN,NaN,161805.36,"AJINOMOTO CO., (THAILAND) LTD.",16800
1,TH100268,NaN,NaN,NaN,NaN,NaN,326154.49,ALUCON PUBLIC COMPANY LIMITED,26229
2,TH100347,NaN,10908.0,NaN,NaN,NaN,15165662.70,"KERRY-APEX (THAILAND) CO., LTD.",558000
3,TH100477,1097602.98,188500.0,NaN,5500.0,NaN,13679531.94,"ASIAN HONDA MOTOR CO.,LTD.",634757
7,TH100707,NaN,NaN,NaN,NaN,NaN,72515.25,BASF (THAI) LIMITED.,41938
...,...,...,...,...,...,...,...,...,...
326,TH511109,5000.00,NaN,NaN,NaN,NaN,11610062.64,SINO LOGISTICS CORPORATION PUBLIC COMPANY LIMITED,123500
327,TH513541,NaN,NaN,NaN,NaN,NaN,72133.46,AGC VINYTHAI PUBLIC COMPANY LIMITED,54000
328,TH513953,NaN,NaN,NaN,NaN,NaN,314890.36,"HYUNDAI GLOVIS LOGISTICS (THAILAND) CO.,LTD",85714
329,TH515918,NaN,NaN,NaN,NaN,NaN,672859.86,"LENZING (THAILAND) CO.,LTD.",231429


In [200]:
df_final.rename(columns={
    'Parent Customer Code': 'Customer Code',
    'Parent Customer Name': 'Customer Name',
    'Parent Credit Limit': 'Credit Limit'
}, inplace=True)

In [201]:
df_final = df_final.fillna(0)

In [202]:
df_final['over due'] = df_final['01-07 days'] + df_final['08-15 days'] + df_final['16-30 days'] + df_final['31-60 days'] + df_final['61-90 days']
df_final['outstanding'] = df_final['over due'] + df_final['Not due']
df_final['%'] = df_final['outstanding']/df_final['Credit Limit']*100
df_final = df_final.sort_values(by='%', ascending=False)

In [204]:
col = ['Customer Code', 'Customer Name', 'Credit Limit', '%', 'Not due', '01-07 days', '08-15 days',
       '16-30 days', '31-60 days', '61-90 days', 'over due', 'outstanding']
df_final = df_final.reindex(columns=col)

In [206]:
df_final.sum()

,0
Customer Code,TH500262TH107545TH103330TH511109TH115587TH1017...
Customer Name,"FREIGHTZEN LOGISTICS, LTD.UBE CHEMICALS (ASIA)..."
Credit Limit,33749143
%,354553.775387
Not due,752836456.75
01-07 days,51578819.91
08-15 days,28956644.27
16-30 days,5939190.79
31-60 days,177557.72
61-90 days,23207.11


In [205]:
df_final.to_excel('utilize credit limit.xlsx', index=False)